In [37]:
!pip install biopython

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
from sklearn.model_selection import train_test_split
import numpy as np

In [40]:
from sklearn.svm import SVC

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
from sklearn.model_selection import KFold

In [43]:
T1_pos_5289 = "/content/drive/My Drive/S.T.U.D.Y/11th Semester/Pattern Lab/Excel to Numpy/Output/3rd_pos_5289/kmer 2.npy"
T1_neg_5289 = "/content/drive/My Drive/S.T.U.D.Y/11th Semester/Pattern Lab/Excel to Numpy/Output/4th_neg_5289/kmer 2.npy"
T1_pos_1000 = "/content/drive/My Drive/S.T.U.D.Y/11th Semester/Pattern Lab/Excel to Numpy/Output/2nd_pos_1000/training_code_normalized_kmer 2.npy"
T1_neg_1000 = "/content/drive/My Drive/S.T.U.D.Y/11th Semester/Pattern Lab/Excel to Numpy/Output/1st_neg_1000/training_code_normalized_kmer 2.npy"

T1_x_train_pos = np.load(T1_pos_5289) 
T1_x_train_neg = np.load(T1_neg_5289) 
T1_x_test_pos = np.load(T1_pos_1000) 
T1_x_test_neg = np.load(T1_neg_1000) 

In [44]:
T2_pos_5289 = "/content/drive/My Drive/S.T.U.D.Y/11th Semester/Pattern Lab/Excel to Numpy/Output/3rd_pos_5289/kmer 3.npy"
T2_neg_5289 = "/content/drive/My Drive/S.T.U.D.Y/11th Semester/Pattern Lab/Excel to Numpy/Output/4th_neg_5289/kmer 3.npy"
T2_pos_1000 = "/content/drive/My Drive/S.T.U.D.Y/11th Semester/Pattern Lab/Excel to Numpy/Output/2nd_pos_1000/training_code_normalized_kmer 3.npy"
T2_neg_1000 = "/content/drive/My Drive/S.T.U.D.Y/11th Semester/Pattern Lab/Excel to Numpy/Output/1st_neg_1000/training_code_normalized_Kmer 3.npy"

T2_x_train_pos = np.load(T2_pos_5289) 
T2_x_train_neg = np.load(T2_neg_5289) 
T2_x_test_pos = np.load(T2_pos_1000) 
T2_x_test_neg = np.load(T2_neg_1000) 

In [45]:
y_train_pos = np.tile(1, 5289) 
y_train_neg = np.tile(0, 5289) 
y_test_pos = np.tile(1, 1000) 
y_test_neg = np.tile(0, 1000) 

In [46]:
print(T2_x_test_neg.shape)
print(y_test_neg.shape)

print(T2_x_train_neg.shape)
print(y_train_neg.shape)


(1000, 64)
(1000,)
(5289, 64)
(5289,)


In [47]:
seed = 40
np.random.seed(seed)

In [48]:
T2_x_training = np.concatenate((T2_x_train_pos, T2_x_train_neg), axis= 0)
T2_x_test = np.concatenate((T2_x_test_pos, T2_x_test_neg), axis = 0)

In [49]:
T1_x_training = np.concatenate((T1_x_train_pos, T1_x_train_neg), axis= 0)
T1_x_test = np.concatenate((T1_x_test_pos, T1_x_test_neg), axis = 0)

In [50]:
x_training = np.concatenate((T1_x_training, T2_x_training), axis= 1)
x_test = np.concatenate((T1_x_test, T2_x_test), axis = 1)

In [51]:
print(x_training.shape)

(10578, 80)


In [52]:
print(T1_x_test.shape)

(2000, 16)


In [53]:
y_training = np.concatenate((y_train_pos, y_train_neg))
y_test = np.concatenate((y_test_pos, y_test_neg))

In [54]:
print(T2_x_test.shape)
print(y_test.shape)

(2000, 64)
(2000,)


In [55]:
print(T2_x_training.shape)

(10578, 64)


In [56]:
def avg(list_value):
    return sum(list_value)/len(list_value)

In [57]:
def crossValidation(c, x_training, y_training, x_test, y_test):
    kf = KFold(n_splits = 5, random_state=seed)
    train_accuracy_avg_list = []
    val_accuracy_avg_list = []
    test_accuracy_list = []
    for i in range(0, len(c)):
        clf = SVC(C = c[i],  random_state = seed)
        val_accuracy_list = []
        train_accuracy_list = []
        count = 1
        for train_index, val_index in kf.split(x_training):
            print("Fold : ",count)
            X_train, X_val = x_training[train_index], x_training[val_index]
            y_train, y_val = y_training[train_index], y_training[val_index]
            
            clf.fit(X_train, y_train)

            y_val_pred = clf.predict(X_val)
            val_accuracy = accuracy_score(y_val, y_val_pred)
            val_accuracy_list.append(val_accuracy)

            y_train_pred = clf.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_accuracy_list.append(train_accuracy)

            count += 1 

        print("done", c[i])
        train_accuracy_avg = avg(train_accuracy_list)
        train_accuracy_avg_list.append(train_accuracy_avg)
        val_accuracy_avg = avg(val_accuracy_list)
        val_accuracy_avg_list.append(val_accuracy_avg)

        y_test_pred = clf.predict(x_test)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_accuracy_list.append(test_accuracy)
    print("all done")
    return train_accuracy_avg_list.copy(), val_accuracy_avg_list.copy(), test_accuracy_list.copy()

In [58]:
def print_accuracy(c, train_list, val_list, test_accuracy_list):
    for i in range(0, len(c)):
        print("c :\t", c[i], "\tTrain Accuracy :\t", train_list[i], "\tVal Accuracy :\t", val_list[i], "\tTest Accuracy :\t", test_accuracy_list[i])

In [59]:
c = [1 , 10, 20, 30, 40, 50, 60, 70, 80, 90]

In [60]:
train_accuracy_avg_list, val_accuracy_avg_list,test_accuracy_list = crossValidation(c.copy(), x_training.copy(), y_training.copy(), x_test.copy(), y_test.copy())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 1
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 10
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 20
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 30
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 40
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 50
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 60
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 70
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 80
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 90
all done


In [61]:
print_accuracy(c.copy(), train_accuracy_avg_list.copy(), val_accuracy_avg_list.copy(), test_accuracy_list.copy())

c :	 1 	Train Accuracy :	 0.999881830213255 	Val Accuracy :	 0.9689032788570253 	Test Accuracy :	 0.854
c :	 10 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 20 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 30 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 40 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 50 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 60 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 70 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 80 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 90 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485


In [62]:
c = [0.50, 0.90, 1.20, 2.20, 3, 4.5, 5.5, 6.5, 8, 9]
train_accuracy_avg_list, val_accuracy_avg_list,test_accuracy_list  = crossValidation(c.copy(), x_training.copy(), y_training.copy(), x_test.copy(), y_test.copy())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 0.5
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 0.9
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 1.2
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 2.2
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 3
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 4.5
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 5.5
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 6.5
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 8
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 9
all done


In [63]:
print_accuracy(c.copy(), train_accuracy_avg_list.copy(), val_accuracy_avg_list.copy(), test_accuracy_list.copy())

c :	 0.5 	Train Accuracy :	 0.9995982204908639 	Val Accuracy :	 0.9624757448596084 	Test Accuracy :	 0.8555
c :	 0.9 	Train Accuracy :	 0.9998345656498614 	Val Accuracy :	 0.968430689064965 	Test Accuracy :	 0.855
c :	 1.2 	Train Accuracy :	 0.999881830213255 	Val Accuracy :	 0.9698484584411464 	Test Accuracy :	 0.8525
c :	 2.2 	Train Accuracy :	 1.0 	Val Accuracy :	 0.9712662725066699 	Test Accuracy :	 0.8475
c :	 3 	Train Accuracy :	 1.0 	Val Accuracy :	 0.971360790465082 	Test Accuracy :	 0.848
c :	 4.5 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 5.5 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 6.5 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 8 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485
c :	 9 	Train Accuracy :	 1.0 	Val Accuracy :	 0.97126627250667 	Test Accuracy :	 0.8485


In [64]:
from sklearn.ensemble import RandomForestRegressor

In [66]:
def model_RandomForestRegressor(n_estimators, max_depth, x_training, y_training, x_test, y_test):
    kf = KFold(n_splits = 5, random_state=seed)
    train_accuracy_avg_list = []
    val_accuracy_avg_list = []
    test_accuracy_list = []
    len_n_estimators = len(n_estimators)
    flag = False
    if(len_n_estimators > 1 and max_depth[0] is None ):
        loop_goes = len_n_estimators
        flag = True
    else:
        loop_goes = len(max_depth)

    for i in range(0, loop_goes):
        if(len_n_estimators > 1 and max_depth[0] is None ):
            regr = RandomForestRegressor(n_estimators = n_estimators[i], max_depth = None, random_state = seed)
        else:
            regr = RandomForestRegressor(n_estimators = n_estimators[0], max_depth = max_depth[i], random_state = seed)

        val_accuracy_list = []
        train_accuracy_list = []
        count = 1
        for train_index, val_index in kf.split(x_training):
            print("Fold : ",count)
            X_train, X_val = x_training[train_index], x_training[val_index]
            y_train, y_val = y_training[train_index], y_training[val_index]
            
            regr.fit(X_train, y_train)
            
            val_accuracy = regr.score(X_val, y_val)
            val_accuracy_list.append(val_accuracy)

            train_accuracy = regr.score(X_train, y_train)
            train_accuracy_list.append(train_accuracy)

            count += 1 
        if(flag):
            print("done", n_estimators[i])
        else :
            print("done", max_depth[i])
        train_accuracy_avg = avg(train_accuracy_list)
        train_accuracy_avg_list.append(train_accuracy_avg)
        val_accuracy_avg = avg(val_accuracy_list)
        val_accuracy_avg_list.append(val_accuracy_avg)

        test_accuracy = regr.score(x_test, y_test)
        test_accuracy_list.append(test_accuracy)
    print("all done")
    return train_accuracy_avg_list.copy(), val_accuracy_avg_list.copy(), test_accuracy_list.copy()

In [67]:
def print_accuracy_n_estimators(n_estimators, train_list, val_list, test_accuracy_list):
    for i in range(0, len(n_estimators)):
        print("n_estimators :\t", n_estimators[i], "\tTrain Accuracy :\t", train_list[i], "\tVal Accuracy :\t", val_list[i], "\tTest Accuracy :\t", test_accuracy_list[i])

In [68]:
n_estimators = [50, 100, 150, 200, 250, 300, 350, 400]
max_depth = [None]
train_accuracy_avg_list, val_accuracy_avg_list, test_accuracy_list  = model_RandomForestRegressor(n_estimators.copy(), max_depth, x_training.copy(), y_training.copy(), x_test.copy(), y_test.copy())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 50
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 100
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 150
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 200
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 250
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 300
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 350
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 400
all done


In [69]:
print_accuracy_n_estimators(n_estimators.copy(), train_accuracy_avg_list.copy(), val_accuracy_avg_list.copy(), test_accuracy_list.copy())

n_estimators :	 50 	Train Accuracy :	 0.9951327782209562 	Val Accuracy :	 0.19157262007052225 	Test Accuracy :	 0.1803224
n_estimators :	 100 	Train Accuracy :	 0.9956240324327569 	Val Accuracy :	 0.1920846054050384 	Test Accuracy :	 0.17525620000000008
n_estimators :	 150 	Train Accuracy :	 0.9957734836224403 	Val Accuracy :	 0.19228020314331554 	Test Accuracy :	 0.1820731555555556
n_estimators :	 200 	Train Accuracy :	 0.9958141368572457 	Val Accuracy :	 0.1922248890842381 	Test Accuracy :	 0.18691595000000005
n_estimators :	 250 	Train Accuracy :	 0.9958404230857607 	Val Accuracy :	 0.19230722337329356 	Test Accuracy :	 0.18823542399999993
n_estimators :	 300 	Train Accuracy :	 0.9958830156106995 	Val Accuracy :	 0.19244271116499095 	Test Accuracy :	 0.19160373333333336
n_estimators :	 350 	Train Accuracy :	 0.9959060966823389 	Val Accuracy :	 0.19239300890573935 	Test Accuracy :	 0.19139774693877543
n_estimators :	 400 	Train Accuracy :	 0.9959223957578811 	Val Accuracy :	 0.192456

In [70]:
n_estimators = [50]
max_depth = [1,5,10,15,20,25,30]
train_accuracy_avg_list, val_accuracy_avg_list, test_accuracy_list  = model_RandomForestRegressor(n_estimators.copy(), max_depth, x_training.copy(), y_training.copy(), x_test.copy(), y_test.copy())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 1
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 5
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 10
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 15
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 20
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 25
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done 30
all done


In [71]:
def print_accuracy_max_depth(max_depth, train_list, val_list, test_accuracy_list):
    for i in range(0, len(max_depth)):
        print("max_depth :\t", max_depth[i], "\tTrain Accuracy :\t", train_list[i], "\tVal Accuracy :\t", val_list[i], "\tTest Accuracy :\t", test_accuracy_list[i])

In [72]:
print_accuracy_max_depth(max_depth.copy(), train_accuracy_avg_list.copy(), val_accuracy_avg_list.copy(), test_accuracy_list.copy())

max_depth :	 1 	Train Accuracy :	 0.19511509388861256 	Val Accuracy :	 0.03238852799281977 	Test Accuracy :	 0.1388996577798064
max_depth :	 5 	Train Accuracy :	 0.8435328337968659 	Val Accuracy :	 0.16509606274484023 	Test Accuracy :	 0.208609290504999
max_depth :	 10 	Train Accuracy :	 0.9787566635278184 	Val Accuracy :	 0.18901026271559707 	Test Accuracy :	 0.16017579472619647
max_depth :	 15 	Train Accuracy :	 0.9929151340691929 	Val Accuracy :	 0.19106053079656998 	Test Accuracy :	 0.1714885843247954
max_depth :	 20 	Train Accuracy :	 0.9949679180679889 	Val Accuracy :	 0.19148841448449289 	Test Accuracy :	 0.17792904248410124
max_depth :	 25 	Train Accuracy :	 0.9951298420528273 	Val Accuracy :	 0.19157262007052225 	Test Accuracy :	 0.1807405749333334
max_depth :	 30 	Train Accuracy :	 0.9951327782209562 	Val Accuracy :	 0.19157262007052225 	Test Accuracy :	 0.1803224
